In [1]:
from Import.FEMReader import *
from Core.GlobalK     import *
from Export.ExportVTK import *

In [2]:
inp_path = r'C:\Users\Михаил\Desktop\Prog\Python\PyFEM\Data\Column_v1.inp'

setup = read_setup(inp_path)

In [3]:
nodes = setup['*NODE']
elems = setup['*ELEMENT, TYPE=C3D4, ELSET=P2;Default PSHELL_Volume_1']

fix_set  = setup['*NSET, NSET=Basement_Set']
load_set = setup['*NSET, NSET=Top_Set']

n_dofs = nodes.shape[0]*3

matprops = [(210e9, 0.3, 7950) for i in range(len(elems))]

In [4]:
# Global Stiffness matrix and load vector assembly
K = assemble_K(nodes, elems, matprops, callback = local_K_tetra)

f = np.zeros(n_dofs)
fixed_dofs = []

for n in fix_set:
    fixed_dofs += [3*n, 3*n+1, 3*n+2]

for n in load_set:
    f[3*n+1] = 1e4
    fixed_dofs += [3*n, 3*n+2]

In [5]:
# Solving for displacements
U = solve_linear(K, f, fixed_dofs)

In [6]:
# Exporting displacements
ux, uy, uz = U[0::3], U[1::3], U[2::3]
set_indicator = np.zeros(nodes.shape[0])
set_indicator[fix_set] = 1
set_indicator[load_set] = 2
export_VTK(setup['*NODE'][:, 1:], setup['*ELEMENT, TYPE=C3D4, ELSET=P2;Default PSHELL_Volume_1'], 'Test.vtk', node_scalars={
    'Displacements' : np.column_stack([ux, uy, uz]), 'Sets': set_indicator
})